# Counting Coreference Information

## 1. Introduction

As explained in NB `2. annotation_aid` the following information is important for coreference annotation:

* Noun Phrases (`NP`)
* Appositions (`Appo`) [not covered in this NB]
* Verbs (`verb`)
* Substantives/Nouns (`subs`)
* Articles (`art`)
* Proper Nouns (`nmpr`)
* Indepedent Personal Pronouns (`prps`)
* Demonstrative Pronouns (`prde`)
* Suffixes that are attached to Prepositional Phrases (`PP`)

This NB gives counts of some of this information to demonstrate how potentially big and complicated the task of coreference resolution in the **Psalms** is. In §5 however you can fill in any Hebrew Bible Book and Chapter to check out other books. 

In this NB the tables with the counts are most important, but the I've also included the accompanying code. The paragraphs indicated below are the table paragraphs: 

* §6.1 gives counts of the features/categories `pgn_prps`, `pgn_prde`, `pgn_verb`, `pgn_prs`. These features give the Person, Gender and Number (PGN) of respectively indepedent personal pronouns, demonstrative pronouns, verbs and suffixes in one 'package' of information instead of three separate features (`ps`, `gn`, `nu`).

* §6.2 gives counts of all possible PGN information within the categories `pgn_prps`, `pgn_prde`, `pgn_verb`, `pgn_prs`. 

* §7.1 gives counts of  phrase types (`typ`) that are determined (`det`), undetermined (`und`) or if not applicable (`NA`). If there are no values for phrase type at all, because they can not be determined, this is indicated by `NaN`. In NB `2. annotation_aid` I stated that for feature `typ` `PP` or `NP` the determination (`det`) is important; in this count NB I have included all phrase types that can contain coreference information (see the enumeration above). These phrase types are: 
    * `VP`: Verbal phrase
    * `NP`: Nominal phrase
    * `PrNP`: Proper-noun phrase
    * `AdvP`: Adverbial phrase
    * `PP`: Prepositional phrase
    * `PPrP`: Personal pronoun phrase
    * `DPrP`: Demonstrative pronoun phrase

* §7.2 gives counts of Phrase Dependent Part of Speech (`pdp`) within phrase types. The `subs` within `NP`'s and `PP`'s are relevant for annotation. The same goes for the verbs in `VP`'s. 

* §8.1 gives counts of the status (`st`) of `sub`'s within the phrase types that are relevant for this study. The status can be: `a` (absolute), `c` (construct) or `e` (emphatic). Not all words have `st` (e.g. adverbs), these are marked as `NA`.

## 2. Import modules and utils

In [1]:
import sys, os, re, pickle, csv, collections
from collections import *
from IPython.display import HTML
from pprint import pprint

from tf.fabric import Fabric
from tf.extra.bhsa import Bhsa
from utils import * 

import pandas as pd

from print_datetime import *
print_datetime()

Notebook last updated by Christiaan at 2018-09-18 18:50:27.971826


## 3. Import Text-Fabric data 

For the coreference data I used the frozen 2017 version, taken from the ETCBC 2017-10-06. The 2017 version is archived in Zenodo: DOI: [doi.org/10.5281/zenodo.1007624](https://zenodo.org/record/1302798#.W5ocuC1g3pI). 

I use 2017 because I don't want the data to change while I'm annotating. The annotated data will be made available through TF later on. 

In [2]:
VERSION = '2017'
DATABASE = '~/github'
BHSA = f'etcbc/bhsa/tf/{VERSION}'
REFERENCE = f'bh-reference-system/tf/{VERSION}'
HEADS = f'etcbc/lingo/heads/tf/{VERSION}'
TF = Fabric(locations=[DATABASE], modules=[BHSA, REFERENCE, HEADS], silent=False )

This is Text-Fabric 5.5.22
Api reference : https://dans-labs.github.io/text-fabric/Api/General/
Tutorial      : https://github.com/Dans-labs/text-fabric/blob/master/docs/tutorial.ipynb
Example data  : https://github.com/Dans-labs/text-fabric-data

122 features found and 0 ignored


## 4. Import Text-Fabric features 

Import the features that are needed for the annotation proces and some extra ones. I always import more features than I need, it saves me errors when I try to use them, but forgot to load them anyway. 

In [3]:
api = TF.load('''
    otype
    lex book chapter verse
    nu ps gn prs ls lex gloss
    function sp typ pdp det st language 
    pgn_prps pgn_prde pgn_verb 
    pgn_verb_prs pgn_prs 
    heads prep_obj 
''', silent=True)

api.makeAvailableIn(globals())

#F, E, T, L = api.F, api.E, api.T, api.L

B = Bhsa(api, 'counts', version=VERSION)

**Documentation:** <a target="_blank" href="https://etcbc.github.io/bhsa" title="{provenance of this corpus}">BHSA</a> <a target="_blank" href="https://etcbc.github.io/bhsa/features/hebrew/2017/0_home.html" title="{CORPUS.upper()} feature documentation">Feature docs</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/Bhsa/" title="BHSA API documentation">BHSA API</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/" title="text-fabric-api">Text-Fabric API 5.5.22</a> <a target="_blank" href="https://dans-labs.github.io/text-fabric/Api/General/#search-templates" title="Search Templates Introduction and Reference">Search Reference</a>

## 5. Fill in Hebrew Bible Book and Chapter

In [4]:
# Set any Hebrew Bible Book
MY_BOOK = {'Psalms'}

# Indicate a range of chapters
MY_CHAPTERS = set(range(1,151))

## 6. Person, Number, Gender Categories in the Psalms

In [5]:
pgn_count_dict = {}
all_pgn_count = {}

for book in F.otype.s('book'):
    book_name = T.bookName(book)
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:
            pgn_count_dict[book_name] = defaultdict(int)
            all_pgn_count[book_name] = defaultdict(int)
            for w in L.d(book, 'word'):
                pgn_prps = F.pgn_prps.v(w)
                pgn_prde = F.pgn_prde.v(w)
                pgn_verb = F.pgn_verb.v(w)
                pgn_prs = F.pgn_prs.v(w)
                pgn_verb_prs = F.pgn_verb_prs.v(w)
                
                # Count all categories of PGN
                if pgn_prps:
                    pgn_count_dict[book_name]['pgn_prps'] += 1
                if pgn_prde:
                    pgn_count_dict[book_name]['pgn_prde'] += 1
                if pgn_verb:
                    pgn_count_dict[book_name]['pgn_verb'] += 1
                if pgn_prs:
                    pgn_count_dict[book_name]['pgn_prs'] += 1
                
                # Count all sorts of PGN within all PGN categories
                for pgn in all_pgn_set:
                
                    if pgn == pgn_prps:
                        all_pgn_count[book_name][pgn] += 1
                    elif pgn not in all_pgn_count[book_name]:
                        all_pgn_count[book_name][pgn] = 0 

                    if pgn == pgn_prde:
                        all_pgn_count[book_name][pgn] += 1
                    elif pgn not in all_pgn_count[book_name]:
                        all_pgn_count[book_name][pgn] = 0 

                    if pgn == pgn_verb:
                        all_pgn_count[book_name][pgn] += 1
                    elif pgn not in all_pgn_count[book_name]:
                        all_pgn_count[book_name][pgn] = 0 

                    if pgn == pgn_prs:
                        all_pgn_count[book_name][pgn] += 1
                    elif pgn not in all_pgn_count[book_name]:
                        all_pgn_count[book_name][pgn] = 0

#pprint(pgn_count_dict)
#pprint(all_pgn_count)

## 6.1 Table: Person, Number, Gender Categories in the Psalms

In [6]:
pgn_count_dict_df = pd.DataFrame.from_dict(pgn_count_dict, orient='index')
pgn_count_dict_df

,pgn_verb,pgn_prs,pgn_prps,pgn_prde
Psalms,5294,4651,294,49


## 6.2 Table: sorts of PGN within all PGN categories in the Psalms

In [18]:
all_pgn_count_df = pd.DataFrame.from_dict(all_pgn_count, orient='index')
all_pgn_count_df

,Csg,P2Fpl,P1Csg,UFsg,Fsg-2,P3Cpl,UFpl,P3Mpl,Fsg,Fsg-3,...,P3Fsg,UUU,P1Cpl,UMpl,P2Mpl,UMsg,Msg,P3Fpl,P2Msg,Cpl-2
Psalms,0,0,2450,23,0,392,6,1184,0,1,...,345,286,343,107,294,305,0,21,2279,8


## 7. Phrase types - Determined and Undetermined

In [8]:
### Create dictionaries with Phrase Types of Choice

phr_typ_count = {}
phr_typ_pdp_count = {}
phr_typ_st_count = {}

for t in {'VP','NP','PrNP', 'AdvP', 'PP', 'PPrP', 'DPrP'}:
        phr_typ_count[t] = defaultdict(int)
        phr_typ_pdp_count[t] = defaultdict(int)
        phr_typ_st_count[t] = defaultdict(int)

In [9]:
for book in F.otype.s('book'):
    book_name = T.bookName(book)
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:

            for phrase in L.d(chn, 'phrase'):
                typ = F.typ.v(phrase)
                det = F.det.v(phrase)
                
                if typ in {'VP','NP','PrNP', 'AdvP', 'PP', 'PPrP', 'DPrP'}:
                    if det:
                         phr_typ_count[typ][det] += 1
                    else:
                        phr_typ_count[typ][det] = 0
#pprint(phr_typ_count)

## 7.1 Table: Phrase types - Determined and Undetermined

In [10]:
phr_typ_count_df = pd.DataFrame.from_dict(phr_typ_count, orient='index')
phr_typ_count_df

,NA,det,und
AdvP,292.0,NaN,NaN
DPrP,NaN,36.0,NaN
NP,NaN,2109.0,2061.0
PP,35.0,2560.0,1145.0
PPrP,NaN,291.0,NaN
PrNP,NaN,585.0,NaN
VP,5294.0,NaN,NaN


## 7.2 Phrase Dependent Part of Speech within Phrase Types

In [11]:
for book in F.otype.s('book'):
    book_name = T.bookName(book)
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:

            for phrase in L.d(chn, 'phrase'):
                typ = F.typ.v(phrase)
                det = F.det.v(phrase)
                
                if typ in {'VP','NP','PrNP', 'AdvP', 'PP', 'PPrP', 'DPrP'}:
                    for w in L.d(phrase, 'word'):
                        pdp = F.pdp.v(w)
                        if pdp:
                            phr_typ_pdp_count[typ][pdp] += 1
#pprint(phr_typ_pdp_count)

## 7.3 Table: Phrase Dependent Part of Speech within Phrase Types

In [12]:
phr_typ_pdp_count_df = pd.DataFrame.from_dict(phr_typ_pdp_count, orient='index')
phr_typ_pdp_count_df

,verb,prep,subs,conj,nmpr,adjv,art,advb,prps,nega,prde,prin,inrg
AdvP,NaN,2.0,5,5.0,NaN,NaN,NaN,298.0,NaN,NaN,NaN,NaN,NaN
DPrP,NaN,NaN,3,NaN,NaN,NaN,1.0,NaN,NaN,NaN,36.0,NaN,NaN
NP,NaN,145.0,5382,235.0,211.0,98.0,221.0,33.0,3.0,3.0,2.0,NaN,NaN
PP,NaN,3910.0,3608,122.0,430.0,69.0,493.0,35.0,NaN,2.0,10.0,12.0,13.0
PPrP,NaN,1.0,16,4.0,2.0,NaN,NaN,5.0,291.0,NaN,NaN,NaN,NaN
PrNP,NaN,3.0,76,12.0,602.0,2.0,4.0,2.0,NaN,NaN,NaN,NaN,NaN
VP,5294.0,268.0,9,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 8. Status of Subs within Phrases of Choice

In [13]:
for book in F.otype.s('book'):
    book_name = T.bookName(book)
    for chn in L.d(book, 'chapter'):
        chapter = F.chapter.v(chn)
        if book_name in MY_BOOK and chapter in MY_CHAPTERS:

            for phrase in L.d(chn, 'phrase'):
                typ = F.typ.v(phrase)
                det = F.det.v(phrase)
                
                if typ in {'VP','NP','PrNP', 'AdvP', 'PP', 'PPrP', 'DPrP'}:
                    for w in L.d(phrase, 'word'):
                        pdp = F.pdp.v(w)
                        st = F.st.v(w)
                        if pdp == 'subs':
                            if st == 'a':
                                phr_typ_st_count[typ]['absolute'] += 1
                            if st == 'c':
                                phr_typ_st_count[typ]['construct'] += 1
                            if st == 'e':
                                phr_typ_st_count[typ]['emphatic'] += 1
#pprint(phr_typ_st_count)

{'AdvP': defaultdict(<class 'int'>, {'absolute': 5}),
 'DPrP': defaultdict(<class 'int'>, {'construct': 1, 'absolute': 2}),
 'NP': defaultdict(<class 'int'>, {'construct': 1020, 'absolute': 4362}),
 'PP': defaultdict(<class 'int'>, {'construct': 793, 'absolute': 2815}),
 'PPrP': defaultdict(<class 'int'>, {'absolute': 12, 'construct': 4}),
 'PrNP': defaultdict(<class 'int'>, {'absolute': 66, 'construct': 10}),
 'VP': defaultdict(<class 'int'>, {'construct': 3, 'absolute': 6})}


## 8.1 Table: Status of Subs within Phrases of Choice

In [15]:
phr_typ_st_count_df = pd.DataFrame.from_dict(phr_typ_st_count, orient='index')
phr_typ_st_count_df

,construct,absolute
AdvP,NaN,5
DPrP,1.0,2
NP,1020.0,4362
PP,793.0,2815
PPrP,4.0,12
PrNP,10.0,66
VP,3.0,6


## 9. Double checking with TF queries 

In [ ]:
query = '''
book book=Psalmi
  verse
    phrase typ=NP
    
'''
res = B.search(query)
B.table(res, linked=3, end=10)

In [ ]:
query = '''
book book=Psalmi
  verse
    phrase typ=PP
'''
res = B.search(query)
B.table(res, linked=3, end=10)

In [ ]:
query = '''
book book=Psalmi
    word pdp=subs
'''
res = B.search(query)
B.table(res, linked=3, end=10)